In [1]:
%pip install bootstrapped

In [2]:
import numpy as np
import pandas as pd
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import bootstrapped
import warnings
warnings.filterwarnings("ignore")

<h3>1 (a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/
Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\
%29 . The dataset contains 7 folders that represent seven types of activities. In
each folder, there are multiple files each of which represents an instant of a human
performing an activity.1 Each file containis 6 time series collected from activities
of the same person, which are called avg rss12, var rss12, avg rss13, var rss13,
vg rss23, and ar rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.<br><br>
1 (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,
2, and 3 in other folders as test data and other datasets as train data.

In [3]:
col_name = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','vg_rss23','ar_rss23']
folder = ['bending1','bending2','cycling','lying','sitting','standing','walking']

In [4]:
df = pd.read_csv("Arem/bending1/dataset1.csv",names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)])
df['activity'] = 'bending1'
instance = 1
for i in folder:
    for j in range(1,4):
        if i == 'bending1':
            if j == 2:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)])
                df1['activity'] = i
                df = df.append(df1, ignore_index=True)
                instance+=1
                
        elif i == 'bending2':
            if j<3:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)])
                df1['activity'] = i
                df = df.append(df1, ignore_index=True)
                instance+=1
        else:
            path = "Arem/"+i+"/dataset"+str(j)+".csv"
            df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)])
            df1['activity'] = i
            df = df.append(df1, ignore_index=True)
            instance+=1

test = df
print("Test data shape: ",test.shape)
print("Number of instances in test data:",instance)
test.head(5)

Test data shape:  (9120, 8)
Number of instances in test data: 19


,time,avg_rss12,var_rss12,avg_rss13,var_rss13,vg_rss23,ar_rss23,activity
0,0,39.25,0.43,22.75,0.43,33.75,1.3,bending1
1,250,39.25,0.43,23.00,0.00,33.00,0.0,bending1
2,500,39.25,0.43,23.25,0.43,33.00,0.0,bending1
3,750,39.50,0.50,23.00,0.71,33.00,0.0,bending1
4,1000,39.50,0.50,24.00,0.00,33.00,0.0,bending1


In [5]:
df = pd.read_csv("Arem/bending1/dataset3.csv",names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)])
df['activity'] = 'bending1'
instance = 1
for i in folder:
    for j in range(3,16):
        if i == 'bending1':
            if j < 8 and j>3:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)])
                df1['activity'] = i
                df = df.append(df1, ignore_index=True)
                instance+=1
                
        elif i == 'bending2':
            if j < 7 and j!=4:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)],usecols = range(7))
                df1['activity'] = i
                df = df.append(df1, ignore_index=True)
                instance+=1
                
            if j == 4:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ' ', skiprows = [i for i in range(0,5)],usecols = range(7))
                df1['activity'] = i
                df = df.append(df1, ignore_index=True)
                instance+=1
                
        else:
            if j > 3:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)])
                df1['activity'] = i
                df = df.append(df1, ignore_index=True)
                instance+=1
                
train = df
print("Train data shape: ",train.shape)
print("Number of instances in training data:",instance)
train.head(5)

Train data shape:  (33119, 8)
Number of instances in training data: 69


,time,avg_rss12,var_rss12,avg_rss13,var_rss13,vg_rss23,ar_rss23,activity
0,0,42.00,0.71,21.25,0.43,30.00,0.00,bending1
1,250,41.50,0.50,20.25,1.48,31.25,1.09,bending1
2,500,41.50,0.50,14.25,1.92,33.00,0.00,bending1
3,750,40.75,0.83,15.75,0.43,33.00,0.00,bending1
4,1000,40.00,0.71,20.00,2.74,32.75,0.43,bending1


<h3> 1 (c) Feature Extraction
Classification of time series usually needs extracting features from them. In this
problem, we focus on time-domain features.<br><br>
i. Research what types of time-domain features are usually used in time series
classification and list them (examples are minimum, maximum, mean, etc).


Answer: Mean, Minimum, Quartiles 1 and 3, Standard deviations, Maximum and Variance

<h3>ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series
in each instance. You are free to normalize/standardize features or use them
directly.

In [6]:
from collections import OrderedDict

In [7]:
data_dict = OrderedDict()
x=0
for i in folder:
    
    for j in range(1,16):
        x+=1
        if i == 'bending1':
            if j < 8:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)],usecols = range(7))
                data_dict[i+"/dataset"+str(j)] = df1
        elif i == 'bending2':
            if j < 7 and j!=4:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)],usecols = range(7))
                data_dict[i+"/dataset"+str(j)] = df1
            if j == 4:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ' ', skiprows = [i for i in range(0,5)],usecols = range(7))
                data_dict[i+"/dataset"+str(j)] = df1
        else:
                path = "Arem/"+i+"/dataset"+str(j)+".csv"
                df1 = pd.read_csv(path,names = col_name, header = None, sep = ',', skiprows = [i for i in range(0,5)],usecols = range(7))
                data_dict[i+"/dataset"+str(j)] = df1        

In [8]:
print("Total Instances loaded: ",len(data_dict))

Total Instances loaded:  88


In [9]:
column_name = ['avg_rss12','var_rss12','avg_rss13','var_rss13','vg_rss23','ar_rss23']
features = ['min','max','mean','median','std','1stQuart','2ndQuart']
x = 1
result = []
l = ['Instance']
for i in column_name:
    for j in features:
        l.append(i+"_"+j+str(x))
    x+=1
result.append(l)
print("Total Features to be extracted: ",len(result[0])-1)

Total Features to be extracted:  42


In [10]:
x=0
for i in data_dict:
    x+=1
    l = [x]
    for j in column_name:
        l.extend([data_dict[i][j].min(),data_dict[i][j].max(),round(data_dict[i][j].mean(),2),round(data_dict[i][j].median(),2),
        round(data_dict[i][j].std(),2), float(data_dict[i][j].quantile([.25]).iloc[0]), float(data_dict[i][j].quantile([.75]).iloc[0])])
    result.append(l)
# resultdf = pd.DataFrame(result[1:], columns = result[0])
# resultdf    

In [11]:
resultdf = pd.DataFrame(result[1:], columns = result[0])
resultdf

,Instance,avg_rss12_min1,avg_rss12_max1,avg_rss12_mean1,avg_rss12_median1,avg_rss12_std1,avg_rss12_1stQuart1,avg_rss12_2ndQuart1,var_rss12_min2,var_rss12_max2,...,vg_rss23_std5,vg_rss23_1stQuart5,vg_rss23_2ndQuart5,ar_rss23_min6,ar_rss23_max6,ar_rss23_mean6,ar_rss23_median6,ar_rss23_std6,ar_rss23_1stQuart6,ar_rss23_2ndQuart6
0,1,37.25,45.00,40.62,40.50,1.48,39.25,42.0000,0.0,1.30,...,2.19,33.0000,36.00,0.00,1.92,0.57,0.43,0.58,0.0000,1.3000
1,2,38.00,45.67,42.81,42.50,1.44,42.00,43.6700,0.0,1.22,...,2.00,32.0000,34.50,0.00,3.11,0.57,0.43,0.60,0.0000,1.3000
2,3,35.00,47.40,43.95,44.33,1.56,43.00,45.0000,0.0,1.70,...,2.00,35.3625,36.50,0.00,1.79,0.49,0.43,0.51,0.0000,0.9400
3,4,33.00,47.75,42.18,43.50,3.67,39.15,45.0000,0.0,3.00,...,3.85,30.4575,36.33,0.00,2.18,0.61,0.50,0.52,0.0000,1.0000
4,5,33.00,45.75,41.68,41.75,2.24,41.33,42.7500,0.0,2.83,...,2.41,28.4575,31.25,0.00,1.79,0.38,0.43,0.39,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,19.50,45.33,33.59,34.25,4.65,30.25,37.0000,0.0,14.67,...,3.28,13.7300,18.25,0.00,8.32,3.26,3.11,1.64,2.0500,4.3225
84,85,19.75,45.50,34.32,35.25,4.75,31.00,38.0000,0.0,13.47,...,3.12,13.5000,17.75,0.00,9.67,3.43,3.20,1.73,2.1575,4.5650
85,86,19.50,46.00,34.55,35.25,4.84,31.25,37.8125,0.0,12.47,...,2.82,14.0000,17.75,0.00,10.00,3.34,3.08,1.66,2.1600,4.3350
86,87,23.50,46.25,34.87,35.25,4.53,31.75,38.2500,0.0,14.82,...,3.13,13.7500,18.00,0.00,9.51,3.42,3.27,1.69,2.1700,4.5000


<h3>iii. Estimate the standard deviation of each of the time-domain features you
extracted from the data. Then, use Python’s bootstrapped or any other
method to build a 90% bootsrap confidence interval for the standard deviation
of each feature.


In [12]:
newresult = resultdf.drop(columns = ['Instance'])
feature_std = newresult.std().tolist()

In [13]:
bootstrap_list = [['Feature','Standard Deviation','Confidence Interval lower bound','Confidence Interval upper bound']]

for i in range(len(newresult.columns)):
    feature_bs = bs.bootstrap(np.array(newresult[newresult.columns[i]]), bs_stats.std, alpha = 0.10)
    l = [newresult.columns[i],feature_std[i],feature_bs.lower_bound,feature_bs.upper_bound]
    bootstrap_list.append(l)

bootstrapdf = pd.DataFrame(bootstrap_list[1:], columns = bootstrap_list[0])
bootstrapdf

,Feature,Standard Deviation,Confidence Interval lower bound,Confidence Interval upper bound
0,avg_rss12_min1,9.569975,8.309421,10.819878
1,avg_rss12_max1,4.394362,3.469145,5.401387
2,avg_rss12_mean1,5.335162,4.785159,5.938981
3,avg_rss12_median1,5.440194,4.845284,6.059301
4,avg_rss12_std1,1.772098,1.587258,1.964307
5,avg_rss12_1stQuart1,6.153590,5.639871,6.697263
6,avg_rss12_2ndQuart1,5.138925,4.412018,5.928981
7,var_rss12_min2,0.000000,0.000000,0.000000
8,var_rss12_max2,5.062729,4.694216,5.471706
9,var_rss12_mean2,1.574314,1.433328,1.738554


<h3>iv. Use your judgement to select the three most important time-domain features
(one option may be min, mean, and max).

Answer: As per the above data the most important time-domain features are Minimum, Maximum and Mean

<h3>2. ISLR 3.7.4

<h3> I collect a set of data (n = 100 observations) containing a single predictor and a quantitative
response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e.
Y = β0 + β1X + β2X2 + β3X3 + epsilon.<br>
(a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0+β1X +epsilon. Consider
the training residual sum of squares (RSS) for the linear regression, and also the training RSS
for the cubic regression. Would we expect one to be lower than the other, would we expect
them to be the same, or is there not enough information to tell? Justify your answer.


Answer (a):<br>
The Cubic regression model might have lower RSS than the linear regression. Because owing to the extra terms in cubic regression, the model would be more flexible and can reduce the RSS.The cubic regression might overfit on training data


<h3>(b) Answer (a) using test rather than training RSS.


Answer (b):<br>
          The Linear Regression model will have lower RSS as compared to the Cubic one. This is because as Cubic regression was more flexible and might have over fitted the training data, the test data would lead to increase in error for cubic model. The Linear model having true relationship will perform better.

<h3>(c) Suppose that the true relationship between X and Y is not linear,
but we don’t know how far it is from linear. Consider the training
RSS for the linear regression, and also the training RSS for the
cubic regression. Would we expect one to be lower than the
other, would we expect them to be the same, or is there not
enough information to tell? Justify your answer.


Answer (c):<br>
The Cubic Regression Model will have lower traning RSS as compared to the linear model. The cubic model have more terms and  hence it has more flexibility to fit the model more accurately than linear. This in turn would reduce the error.

<H3>(d) Answer (c) using test rather than training RSS.

Answer (d):<br>
The question does not provide adequate information to suggest which model will have lower Test RSS. This because the test error depends on the kind of relationship between X and Y. If it is more near to cubic then cubic will have lower RSS and the opposite will happen if it is more close to linear.